In [ ]:
#! /usr/bin/env python

###################################################################################
# Copyright   2015, Pittsburgh Supercomputing Center (PSC).  All Rights Reserved. #
# =============================================================================== #
#                                                                                 #
# Permission to use, copy, and modify this software and its documentation without #
# fee for personal use within your organization is hereby granted, provided that  #
# the above copyright notice is preserved in all copies and that the copyright    #
# and this permission notice appear in supporting documentation.  All other       #
# restrictions and obligations are defined in the GNU Affero General Public       #
# License v3 (AGPL-3.0) located at http://www.gnu.org/licenses/agpl-3.0.html  A   #
# copy of the license is also provided in the top level of the source directory,  #
# in the file LICENSE.txt.                                                        #
#                                                                                 #
###################################################################################

import logging

import os.path
from phacsl.utils.collections.phacollections import SingletonMetaClass
import pyrheautils
from collections import deque, defaultdict
from facilitybase import CareTier
from tools_util import readModelInputs, getFacDict
import yaml

runDesc = '/home/welling/git/pyRHEA_github/src/sim/week_run_OC.yaml'

inputDict = readModelInputs(runDesc)
facDict = getFacDict(inputDict)

# Drop annoying community entries
dropL = []
for abbrev, rec in facDict.items():
    if rec['category'] == 'COMMUNITY':
        dropL.append(abbrev)
for abbrev in dropL:
    del facDict[abbrev]

print facDict.keys()



In [ ]:
import phacsl.utils.formats.csv_tools as csv_tools
import phacsl.utils.formats.yaml_tools as yaml_tools
from pyrheautils import pathTranslate

snfDspFname = 'Hospital_and_NH_Disposition_Outcomes_for_RHEA_2.0_-_Adult_Only_-_UPDATED_-_11-15-2017_NH_Outcomes.csv'

snfPath = pathTranslate('$(MODELDIR)/%s' % snfDspFname)
with open(snfPath, 'rU') as f:
    snfKeys, snfRecs = csv_tools.parseCSV(f)
print snfKeys
snfCsvD = {r['NH']: r for r in snfRecs}

In [ ]:
with open(pathTranslate('$(MODELDIR)/fac_to_cat_marginalized_direct.yaml'), 'rU') as f:
    facToCatD = yaml.load(f)
print facToCatD.keys()

In [ ]:
newFacDict = {}
directMtxFname = 'OC_Direct_Transfer_Matrices_for_RHEA_2.0_-_Adult_Only_-_09-15-2017_FINAL.xlsx'
transProv = ('(%s summed by dest category and normalized) * (%s total discharges * frac to NH or HOSP)'
             % (directMtxFname, snfDspFname))

for abbrev, rec in facDict.items():
    newR = rec.copy()
    if rec['category'] == 'NURSINGHOME':
        if abbrev in snfCsvD:
            csvR = snfCsvD[abbrev]
            totDsch = csvR['Total Annual Discharges']
            totTrans = totDsch * (1.0 - (csvR[u'(4) Discharged to Home**'] + csvR[u'(1) Patient Died']))
            
            tNHFrac = facToCatD[abbrev]['NURSINGHOME'] if 'NURSINGHOME' in facToCatD[abbrev] else 0.0
            tLTACFrac = facToCatD[abbrev]['LTAC'] if 'LTAC' in facToCatD[abbrev] else 0.0
            tHospFrac = facToCatD[abbrev]['HOSPITAL'] if 'HOSPITAL' in facToCatD[abbrev] else 0.0
            newR['totalDischarges'] = {'value': totDsch,
                                      'prov': '%s col B' % snfDspFname}
            
            newR['totalTransfersOut'] = [
                {'category': 'NURSINGHOME', 'count': {'value': totTrans*tNHFrac, 'prov': transProv}},
                {'category': 'LTAC', 'count': {'value': totTrans*tLTACFrac, 'prov': transProv}},
                {'category': 'HOSPITAL', 'count': {'value': totTrans*tHospFrac, 'prov': transProv}}
            ]
        else:
            print 'No data for %s' % abbrev

    newFacDict[abbrev] = newR
yaml_tools.save_all(pathTranslate('$(MODELDIR)/facilityfactsUpdated'), newFacDict.values())